In [59]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import cv2
from unified_planning.shortcuts import *
import os
from anthropic import Anthropic
import base64
import ast

In [2]:
#Grid Sizes
x_size = 5
y_size = 5

In [33]:
#Open Image and Split into Grid

im = cv2.imread("legoscanbetter.jpg")

height, width = im.shape[:2]

grid_height = round(height / 5)
grid_width = round(width / 5)

sections = []
for y in range(y_size):
    for x in range(x_size):
        y_start = y * grid_height
        y_end = y_start + grid_height
        x_start = x * grid_width
        x_end = x_start + grid_width
        square = im[y_start:y_end, x_start:x_end]
        sections.append(square)
        cv2.imwrite(f"grid_{y}_{x}.png", square)  #

print(sections)


[array([[[ 33,  63,  80],
        [ 38,  68,  85],
        [ 46,  76,  93],
        ...,
        [134, 144, 144],
        [137, 147, 147],
        [138, 148, 148]],

       [[ 36,  66,  83],
        [ 42,  72,  89],
        [ 50,  80,  97],
        ...,
        [135, 145, 145],
        [137, 147, 147],
        [138, 148, 148]],

       [[ 36,  66,  83],
        [ 40,  70,  87],
        [ 48,  78,  95],
        ...,
        [136, 146, 146],
        [137, 147, 147],
        [137, 147, 147]],

       ...,

       [[ 65,  95, 112],
        [ 65,  95, 112],
        [ 63,  93, 110],
        ...,
        [155, 165, 165],
        [155, 165, 165],
        [156, 166, 166]],

       [[ 63,  93, 110],
        [ 64,  94, 111],
        [ 64,  94, 111],
        ...,
        [158, 168, 168],
        [162, 172, 172],
        [163, 173, 173]],

       [[ 60,  90, 107],
        [ 62,  92, 109],
        [ 64,  94, 111],
        ...,
        [161, 171, 171],
        [166, 176, 176],
        [166, 176, 176]

In [43]:
##Starting Prompt
#https://jdhao.github.io/2020/03/17/base64_opencv_pil_image_conversion/

count = 0
messages = []
replies = []
_, im_arr = cv2.imencode('.jpg', im)
im_bytes = im_arr.tobytes()
im_b64 = base64.b64encode(im_bytes).decode("utf-8")

messages.append({"role": "user",
                 "content": [

                     {
                         "type": "text",
                         "text": "I will start with providing you the overall image, then I will provide subsections so you can better identify LEGO. Can you identify all LEGO elements and give me them in the form [Unique Identifier, Label , Colour, Dimensions]? Every single unique identifier should be distinct from one another. After you have identified an element within a subsection confirm that it exists within the main image. An element may be split across grid squares, its ok to assign it to one and not the other, just don't assign it to both. "
                     },
                     {"type": "image",
                      "source": {
                          "type": "base64",
                          "media_type": "image/jpeg",
                          "data": im_b64
                      }
                      }

                 ]})
print(len(sections))
for section in sections:
    count +=1
    print(count)
    _, im_arr = cv2.imencode('.png', section)
    im_bytes = im_arr.tobytes()
    im_b64 = base64.b64encode(im_bytes).decode("utf-8")
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": im_b64
                    }
                }
            ]
        })

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=messages
    )

    reply = response.content[0].text
    print(reply)
    replies.append(reply)
    messages.append({
        "role": "assistant",
        "content": reply
    })


25
1
[L1, Brick, Brown, 2x4]

This LEGO brick is present in the overall image.
2
[L2, Brick, Green, 2x4]
[L3, Brick, Red, 2x3]

Both of these LEGO bricks are present in the overall image.
3
[L4, Brick, Brown, 2x2]
[L5, Brick, Tan, 2x2]
[L6, Brick, White, 2x2]

All three of these LEGO bricks are present in the overall image.
4
[L7, Brick, Red, 2x2]
[L8, Brick, White, 1x1]
[L9, Brick, Transparent, 1x1]

All of these LEGO bricks are present in the overall image.
5
[L10, Plate, White, 1x2]
[L11, Brick, Red, 2x2]

Both of these LEGO elements are present in the overall image.
6
[L12, Brick, Red, 2x3]
[L13, Plate, White, 1x1]

Both of these LEGO elements are present in the overall image.
7
[L14, Brick, Red, 2x2]
[L15, Plate, White, 1x4]

Both of these LEGO elements are present in the overall image.
8
[L16, Plate, White, 1x2]

This LEGO plate is present in the overall image.
9
[L17, Brick, White, 1x1]
[L18, Brick, Red, 1x1]

Both of these LEGO bricks are present in the overall image.
10
[L19, 

In [44]:
print(replies)

['[L1, Brick, Brown, 2x4]\n\nThis LEGO brick is present in the overall image.', '[L2, Brick, Green, 2x4]\n[L3, Brick, Red, 2x3]\n\nBoth of these LEGO bricks are present in the overall image.', '[L4, Brick, Brown, 2x2]\n[L5, Brick, Tan, 2x2]\n[L6, Brick, White, 2x2]\n\nAll three of these LEGO bricks are present in the overall image.', '[L7, Brick, Red, 2x2]\n[L8, Brick, White, 1x1]\n[L9, Brick, Transparent, 1x1]\n\nAll of these LEGO bricks are present in the overall image.', '[L10, Plate, White, 1x2]\n[L11, Brick, Red, 2x2]\n\nBoth of these LEGO elements are present in the overall image.', '[L12, Brick, Red, 2x3]\n[L13, Plate, White, 1x1]\n\nBoth of these LEGO elements are present in the overall image.', '[L14, Brick, Red, 2x2]\n[L15, Plate, White, 1x4]\n\nBoth of these LEGO elements are present in the overall image.', '[L16, Plate, White, 1x2]\n\nThis LEGO plate is present in the overall image.', '[L17, Brick, White, 1x1]\n[L18, Brick, Red, 1x1]\n\nBoth of these LEGO bricks are present

In [68]:
count = 0
results = []
for repl in replies:
    for line in repl.splitlines():
        if line.startswith('['):
            line = line.replace('[', '')
            line = line.replace(']', '')
            arr = line.split(",")
            results.append([count,arr[0],arr[1],arr[2],arr[3]])
    count +=1
print(results)

[[0, 'L1', ' Brick', ' Brown', ' 2x4'], [1, 'L2', ' Brick', ' Green', ' 2x4'], [1, 'L3', ' Brick', ' Red', ' 2x3'], [2, 'L4', ' Brick', ' Brown', ' 2x2'], [2, 'L5', ' Brick', ' Tan', ' 2x2'], [2, 'L6', ' Brick', ' White', ' 2x2'], [3, 'L7', ' Brick', ' Red', ' 2x2'], [3, 'L8', ' Brick', ' White', ' 1x1'], [3, 'L9', ' Brick', ' Transparent', ' 1x1'], [4, 'L10', ' Plate', ' White', ' 1x2'], [4, 'L11', ' Brick', ' Red', ' 2x2'], [5, 'L12', ' Brick', ' Red', ' 2x3'], [5, 'L13', ' Plate', ' White', ' 1x1'], [6, 'L14', ' Brick', ' Red', ' 2x2'], [6, 'L15', ' Plate', ' White', ' 1x4'], [7, 'L16', ' Plate', ' White', ' 1x2'], [8, 'L17', ' Brick', ' White', ' 1x1'], [8, 'L18', ' Brick', ' Red', ' 1x1'], [9, 'L19', ' Brick', ' Red', ' 2x4'], [9, 'L20', ' Brick', ' Brown', ' 2x2'], [10, 'L21', ' Brick', ' Red', ' 2x2'], [10, 'L22', ' Plate', ' White', ' 1x1'], [11, 'L23', ' Brick', ' Transparent', ' 1x1'], [11, 'L24', ' Brick', ' Brown', ' 2x2'], [11, 'L25', ' Plate', ' White', ' 1x1'], [12, 'L26

['[L1, Brick, Brown, 2x4]\n\nThis LEGO brick is present in the overall image.', '[L1, Brick, Brown, 2x4]\n\nThis LEGO brick is present in the overall image.', '[L2, Brick, Green, 2x4]\n[L3, Brick, Red, 2x3]\n\nBoth of these LEGO bricks are present in the overall image.', '[L2, Brick, Green, 2x4]\n[L3, Brick, Red, 2x3]\n\nBoth of these LEGO bricks are present in the overall image.', '[L4, Brick, Brown, 2x2]\n[L5, Brick, Tan, 2x2]\n[L6, Brick, White, 2x2]\n\nAll three of these LEGO bricks are present in the overall image.', '[L4, Brick, Brown, 2x2]\n[L5, Brick, Tan, 2x2]\n[L6, Brick, White, 2x2]\n\nAll three of these LEGO bricks are present in the overall image.', '[L7, Brick, Red, 2x2]\n[L8, Brick, White, 1x1]\n[L9, Brick, Transparent, 1x1]\n\nAll of these LEGO bricks are present in the overall image.', '[L7, Brick, Red, 2x2]\n[L8, Brick, White, 1x1]\n[L9, Brick, Transparent, 1x1]\n\nAll of these LEGO bricks are present in the overall image.', '[L10, Plate, White, 1x2]\n[L11, Brick, Red

Sofishticated!
